<a href="https://colab.research.google.com/github/ShrutiParate/GEN_AI_Digitide/blob/main/Week1/Day4_Basics_of_RAG/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers sentence-transformers faiss-cpu PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, glob, json, zipfile
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from PyPDF2 import PdfReader

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Chunking function
def chunk_text(text, chunk_size=800, overlap=200):
    text = text.strip()
    if not text:
        return []
    chunks = []
    start = 0
    L = len(text)
    while start < L:
        end = min(start + chunk_size, L)
        chunks.append(text[start:end].strip())
        start += chunk_size - overlap
    return [c for c in chunks if len(c) > 20]

# File reading
def read_txt(path):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    except:
        return ""

def read_pdf(path):
    try:
        reader = PdfReader(path)
        pages = [p.extract_text() or "" for p in reader.pages]
        return "\n".join(pages)
    except:
        return ""


Device: cuda


In [ ]:
from google.colab import files
import zipfile
import os

print("Please upload your ZIP file containing documents (.txt, .md, .pdf)")
uploaded = files.upload()

# Get uploaded file name
uploaded_zip = list(uploaded.keys())[0]  # First uploaded file
target_dir = "/content/my_docs"
os.makedirs(target_dir, exist_ok=True)

# Extract ZIP
with zipfile.ZipFile(uploaded_zip, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

print(f"Files extracted to: {target_dir}")
!ls -R {target_dir}


Please upload your ZIP file containing documents (.txt, .md, .pdf)


Saving data.zip to data (1).zip
Files extracted to: /content/my_docs
/content/my_docs:
data  myfolder

/content/my_docs/data:
'cloud computing training.pdf'	 Python_refrence.pdf

/content/my_docs/myfolder:
'cloud computing training.pdf'	 Python_refrence.pdf  'refund policy.pdf'


In [ ]:
# This assumes you've already uploaded data.zip to /content/
uploaded_zip = "/content/data.zip"
target_dir = "/content/my_docs"
os.makedirs(target_dir, exist_ok=True)

with zipfile.ZipFile(uploaded_zip, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

print(f"Files extracted to: {target_dir}")
!ls -R {target_dir}


Files extracted to: /content/my_docs
/content/my_docs:
data  myfolder

/content/my_docs/data:
'cloud computing training.pdf'	 Python_refrence.pdf

/content/my_docs/myfolder:
'cloud computing training.pdf'	 Python_refrence.pdf  'refund policy.pdf'


In [ ]:
DOCS_PATH = target_dir
docs = []

for ext in ("*.txt", "*.md", "*.pdf"):
    for path in glob.glob(os.path.join(DOCS_PATH, "**", ext), recursive=True):
        text = read_txt(path) if path.lower().endswith((".txt", ".md")) else read_pdf(path)
        if text and len(text.strip()) > 50:
            docs.append({"title": os.path.basename(path), "text": text})

print(f"Loaded {len(docs)} documents from {DOCS_PATH}")
for d in docs[:5]:
    print("-", d["title"], "(", len(d["text"]), "chars )")


Loaded 5 documents from /content/my_docs
- refund policy.pdf ( 1712 chars )
- Python_refrence.pdf ( 54168 chars )
- cloud computing training.pdf ( 25572 chars )
- Python_refrence.pdf ( 54168 chars )
- cloud computing training.pdf ( 25572 chars )


In [ ]:
metas = []
texts = []
for doc in docs:
    chunks = chunk_text(doc["text"], chunk_size=800, overlap=200)
    for i, c in enumerate(chunks):
        metas.append({"title": doc["title"], "chunk_id": i, "text": c})
        texts.append(c)

print("Total chunks:", len(texts))
if len(texts) == 0:
    print("No chunks found — check DOCS_PATH, file formats, or chunker settings.")


Total chunks: 271


In [ ]:
print("Loading embedding model...")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

batch_size = 32
embs = []
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    emb = embed_model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
    embs.append(emb)
embs = np.vstack(embs).astype("float32")
dim = embs.shape[1]
print("Embeddings shape:", embs.shape)

index = faiss.IndexFlatL2(dim)
index.add(embs)
print("FAISS index vectors:", index.ntotal)


Loading embedding model...
Embeddings shape: (271, 384)
FAISS index vectors: 271


In [ ]:
def retrieve(query, top_k=3):
    q_emb = embed_model.encode([query], convert_to_numpy=True)
    D, I = index.search(q_emb, top_k)
    results = []
    for idx in I[0]:
        if idx < len(metas):
            results.append(metas[idx])
    return results


In [ ]:
def build_prompt(query, contexts, tokenizer, reserved_output_tokens=128):
    header = (
        "Answer the question based on the following context. "
        "If the answer is not directly in the context, you may still try to answer from your knowledge.\n\n"
    )
    ctx_text = "\n\n".join([f"[{i+1}] {c['text']}" for i, c in enumerate(contexts)])
    prompt = header + ctx_text + "\n\nQuestion: " + query
    max_len = tokenizer.model_max_length or 512
    tokens = tokenizer.encode(prompt, truncation=False)
    if len(tokens) > max_len - reserved_output_tokens:
        prompt = tokenizer.decode(tokens[: max_len - reserved_output_tokens])
    return prompt


In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


In [ ]:
def answer(query, top_k=3, max_new_tokens=128):
    contexts = retrieve(query, top_k=top_k)
    prompt = build_prompt(query, contexts, tokenizer)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length).to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text, contexts


In [ ]:
def answer(query, top_k=3, max_new_tokens=128):
    contexts = retrieve(query, top_k=top_k)
    prompt = build_prompt(query, contexts, tokenizer)

    # Limit token length
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)

    text = tokenizer.decode(out[0], skip_special_tokens=True)
    text = " ".join(text.split())  # remove newlines and extra spaces
    return text, contexts

# Test
query = "what is the refund policy?"
ans, ctxs = answer(query, top_k=5)
print("Q:", query)
print("A:", ans)
print("\n--- Contexts ---")
for i, c in enumerate(ctxs):
    clean_text = " ".join(c['text'].split())  # flatten vertical text
    print(f"[{i+1}] {clean_text[:200]}...")


Q: what is the refund policy?
A: tax refund or additional burden

--- Contexts ---
[1] no separate refund mechanism disclosed for this demerger. The process is standard: eligible shareholders receive their respective shares. It is tax-neutral, so there’s no tax refund or additional burd...
[2] ived 1 share each of Digitide and Bluspring for every Quess share held. Listing & Market Debut Digitide opened around ■245, with strong early gains. Bluspring debuted around ■89 but saw early declines...
[3] for verifying access to S3 buckets from outside your AWS account. An ACL is specific to each bucket. You can utilize S3 Object Ownership, an Amazon S3 bucket -level feature, to manage who owns the obj...
[4] for verifying access to S3 buckets from outside your AWS account. An ACL is specific to each bucket. You can utilize S3 Object Ownership, an Amazon S3 bucket -level feature, to manage who owns the obj...
[5] his is a cheaper data -storage class and as the name suggests, this class is best